# Understanding Data


- Coropleta con el tiempo promedio de vuelta en el circuito para ver cuáles son las más tardadas
- Seleccionando un país, te da la distribución de tiempo por vuelta ---> Violin plot
- Desgaste de llantas por circuito
- Coropletas de los países por número de pilotos
- Steamgraph de los puntos obtenidos por cada equipo a lo largo del tiempo ---> Streamgraph
- Seleccionando un equipo ---> barplot con los puntos en cada circuito
                          ---> puntos por circuito
                          ---> nacionalidad de los pilotos
- Histórico, teniendo un filtro de año, una coropleta que pinte el circuito del color del equipo que más puntos ha ganado en él

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Este tiene la ubicación de los circuitos
# longitud y latitud
circuitos = pd.read_csv('../proytecto/SEASON2022/circuits.csv')
circuitos.head(3)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.7380,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_Internati...


In [3]:
races = pd.read_csv('datos/races.csv', usecols=['raceId', 'year'])
puntos = pd.read_csv('datos/puntosequipo.csv', usecols=['team', 'points', 'raceId'])

In [4]:
auxiliar_stream_graph = (puntos
 .merge(races, on='raceId')
 .drop(columns=['raceId'])
 .groupby(['year', 'team'])
 .agg(puntos=pd.NamedAgg('points', np.sum))
 .reset_index()
 .assign(
     puntos_anio=lambda df: df.groupby('year').puntos.transform(np.sum),
     porcentaje=lambda x: np.round(x.puntos / x.puntos_anio * 100, 1)
 )
 .drop(columns=['puntos_anio'])
)
# auxiliar_stream_graph.to_csv('datos/data_for_streamgraph.csv', index=False)

### Ahora vamos a preparar los datos para poder filtrarlos por año, equipo y circuito

In [5]:
races = pd.read_csv('datos/races.csv') #, usecols=['raceId', 'year'])
puntos = pd.read_csv('datos/puntosequipo.csv') #, usecols=['team', 'points', 'raceId'])

In [28]:
# Los datos que necesito son los siguientes:
# año, circuito, equipo, puntos
# año: races -> year
# circuito: races -> name, circuitId
# equipo: puntos -> team
# puntos: puntos -> points
aux_puntos = puntos[['raceId', 'team', 'points']]
aux_races = races[['raceId', 'circuitId', 'year']]
auxiliar_puntos_por_anio = (aux_puntos
 .merge(aux_races, on='raceId', how='inner')
 .sort_values(by='team')
 # Supongamos lo siguiente: se pide de los años 2020 a 2022
 #.query('year >= 2020 and year <= 2022')
 #.groupby(['circuitId', 'team'])
 #.agg(total=pd.NamedAgg('points', np.sum))
 #.reset_index()
 #.groupby('circuitId')
 #.total
 #.max()
)
auxiliar_puntos_por_anio.to_csv('./datos/points_per_year.csv', index=False)

In [31]:
# Vamos a crear un dataframe con los colores de los equipos
# Así va a ser más fácil colorearlos después, que sino es mucho pedo
equipos = auxiliar_puntos_por_anio.team.unique()
colores = ["#C92D4B", "#5E8FAA", "#2293D1", "#358C75", "#F91536", 
           "#B6BABD", "#F58020", "#6CD3BF", "#3671C6", "#37BEDD"]
(pd
 .DataFrame({'team':equipos, 'color':colores})
 .to_csv('datos/colores_equipos.csv', index=False)
)